## Based on the stuff done previously:
* Writes a function that answers questions from multiple reference txt'

As torch is currently not working on my system I can't show the outputs. These do work THO.

In [ ]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer

In [ ]:
def question_answer(corpus_path):
    """
    Answers questions from multiple reference txt's
    
    Parameters:
        corpus_path -> path of the corpus file
    """
    
    while True:    
        global A
        A = input("Q: ")
        exit = ["exit", "quit", "goodbye", "bye"]

        if A.lower() in exit:
            print("A: Goodbye")
            break
        reference = semantic_search(corpus_path, A)
        answer = question_answerer(A, reference)
        
        if answer is None:
            print("A: Apologies, I do not understand the question given. Please try again.")
        else:
            print("A: ", answer)

def semantic_search(corpus_path, sentence):
    """
    Performs semantic search on a corpus.
    
    Parameters:
        corpus_path -> path to the corpus of reference documents.
        sentence -> the sentence to be semantic searched.
        
    Returns:
        the reference text similar to sentence.
    """
    document = [sentence]
    
    # Loading and reading the data
    for filename in os.listdir(corpus_path):
        if filename.endswith(".md") is False:
            continue
        with open(corpus_path + "/" + filename, "r", encoding="utf-8") as file:
            document.append(file.read())
            
    model = hub.load(
        "https://tfhub.dev/google/universal-sentence-encoder-large/5"
    ) # This is the USE model from tensorflow hub. it is designed to semantically produce meaningful vectors
    
    embeddings = model(document)
    
    correlation = np.inner(embeddings, embeddings) # Finding Correlations in the embeddings of the document
    
    closest = np.argmax(correlation[0, 1:]) # Finding the closest document in similarity
    
    similar = document[closest + 1] # Find the most similar document of the bunch
    
    return similar

def question_answerer(question, reference):
    """
    Finds a snippet of text within a document in order to answer questions.
    """
    token = BertTokenizer.from_pretrained(
        'bert-large-uncased-whole-word-masking-finetuned-squad')
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")
    
    question_tokens = token.tokenize(question)
    reference_tokens = token.tokenize(reference)
    
    tokens = ["[CLS]"] + question_tokens + ["[SEP]"] + reference_tokens + ["[SEP]"]
    
    input_id = token.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    input_type_ids = ([0] * (
        1 + len(question_tokens) + 1) +
                      [1] * (len(reference_tokens) + 1
    ))
    
    input_id, input_mask, input_type_ids = map(
        lambda t: tf.expand_dims(
            tf.convert_to_tensor(t, dtype=tf.int32), 0),
        (input_id, input_mask, input_type_ids)
        )
    
    outputs = model(input_ids=input_id, input_mask=input_mask, token_type_ids=input_type_ids)
    
    short_start = tf.argmax(outputs[0][0][1:]) + 1
    short_end = tf.argmax(outputs[1][0][1:]) + 1
    answer_token = tokens[short_start : short_end + 1]
    answer = token.convert_tokens_to_string(answer_token)
    
    if answer == None or answer == "" or question in answer:
        return None

    return answer